# Preprocesado de datasets
En este cuaderno, se encarga de convertir todas las imágenes a formto `.png` con una estructura de carpetas homogénea. Se trabaja con dos conjuntos de datos:
  - **Conjunto de entrenamiento**: imágenes del dataset NHANES II (radiografías y respectivas máscaras).
  - **Conjunto clínico externo**: imágenes de rayos X proporcionadas por el Dr. Julio Manuel Plata Bello (sin máscaras).

Se asegura que todas las imágenes estén en el mismo formato para su uso posterior en el entrenamiento de modelos y en el análisis clínico.

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import pickle as pkl

## Configuración de rutas y funciones de utilidad
Se definen rutas de entrada y salida, así como funciones auxiliares para convertir imágenes `.pkl` y `.jpg` a `.png`.

In [ ]:
ruta_original = 'dataset_original' # Carpeta donde se encuentran las imagenes originales
ruta_procesado = 'dataset_procesado' # Carpeta donde se guardaran las imagenes procesadas

con_mascara = 'dataset_con_mascara' # Carpeta donde se encuentran las imagenes con mascara
sin_mascara = 'dataset_sin_mascara' # Carpeta donde se encuentran las imagenes sin mascara

# Subcarpetas
imagenes = 'imagenes'
mascaras = 'mascaras'

cervical = 'cervical'
lumbar = 'lumbar'
columna = 'columna_completa'

### Conversión de imágenes
Se procesan las imágenes de entrenamiento (con máscara) y las de prueba clínica (sin máscara), guardando los resultados en carpetas separadas bajo una estructura coherente basada en regiones, lo que permitirá su uso en los tres modelos que se probarán.

#### Preprocesado del dataset NHANES II
Las imágenes radiográficas se convierten de `.pkl` a `.png`.

Las máscaras también se pasan a `.png` pero, los píxeles toman valor binario:
  - 0: fondo
  - 1: vértebra

In [3]:
def pkl_2_png_con_mascara(ruta_entrada, ruta_salida):
  cervical_img = os.path.join(ruta_salida, 'cervical', imagenes)
  cervical_masc = os.path.join(ruta_salida, 'cervical', mascaras)
  lumbar_img = os.path.join(ruta_salida, 'lumbar', imagenes)
  lumbar_masc = os.path.join(ruta_salida, 'lumbar', mascaras)

  rutas_salida = [cervical_img, cervical_masc, lumbar_img, lumbar_masc] 

  for ruta in rutas_salida:
    if not os.path.exists(ruta):
      os.makedirs(ruta)
  
  for nombre_fichero in os.listdir(ruta_entrada):
    if nombre_fichero.endswith('.pkl'):
      ruta_imagen = ''
      ruta_mascara = ''
      if 'cervical' in nombre_fichero.lower().split('_'):
        ruta_imagen = cervical_img
        ruta_mascara = cervical_masc
      else:
        ruta_imagen = lumbar_img
        ruta_mascara = lumbar_masc

      ruta_fichero = os.path.join(ruta_entrada, nombre_fichero)

      with open(ruta_fichero, 'rb') as fichero:
        dato = pkl.load(fichero)

      imagen = dato['image']
      mascara = dato['masks']['vertebrae']

      # Guardamos la imagen
      png_imagen = Image.fromarray(imagen)
      ruta_png_imagen = os.path.join(ruta_imagen, nombre_fichero.replace('_masks_4.pkl', '.png'))
      png_imagen.save(ruta_png_imagen)

      # Guardamos la máscara binaria
      png_mascara = Image.fromarray((mascara).astype(np.uint8)) # Convertimos a uint8 para que se guarde correctamente
      ruta_png_mascara = os.path.join(ruta_mascara, nombre_fichero.replace('_masks_4.pkl', '.png'))
      png_mascara.save(ruta_png_mascara)

In [4]:
# Ruta de los datos originales
dataset = 'NHANESII_Annotations_VertebralOsetophytes/nhanes'
ruta_entrada = os.path.join(ruta_original, dataset)

# Rutas de salida de las imágenes y máscaras
ruta_salida = os.path.join(ruta_procesado, con_mascara)

# Convertimos los ficheros pkl a png
pkl_2_png_con_mascara(ruta_entrada, ruta_salida)

#### Preprocesado de dataset Rxs
En este caso, solo se dispone de imágenes radiográficas `.jpg` sin máscara, estas se usaran solo para realizar pruebas y comprobaciones.
El preprocesado que se debe realizar es, cambiar su formato a `.png`.

In [12]:
def jpg_2_png(ruta_entrada, ruta_salida, es_gris=True):
  if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

  for nombre_fichero in os.listdir(ruta_entrada):
    if nombre_fichero.endswith('.jpg'):
      ruta_fichero = os.path.join(ruta_entrada, nombre_fichero)
      
      if es_gris:
        imagen = cv2.imread(ruta_fichero, cv2.IMREAD_GRAYSCALE)
      else:
        pil_imagen = Image.open(ruta_fichero).convert('L') # Convertimos a escala de grises
        imagen = np.array(pil_imagen).astype(np.uint8)
        imagen = cv2.cvtColor(imagen, cv2.COLOR_GRAY2BGR)

      if imagen is not None:
        png_imagen = Image.fromarray(imagen)
        ruta_imagen = os.path.join(ruta_salida, nombre_fichero.replace('.jpg', '.png'))
        png_imagen.save(ruta_imagen)

In [ ]:
# Ruta de los datos originales
dataset = 'Rxs'
ruta_entrada_cervical = os.path.join(ruta_original, dataset, cervical)
ruta_entrada_lumbar = os.path.join(ruta_original, dataset, lumbar)
ruta_entrada_columna = os.path.join(ruta_original, dataset, columna)

# Rutas de salida de las imágenes y máscaras
ruta_salida_cervical = os.path.join(ruta_procesado, sin_mascara, cervical, imagenes)
ruta_salida_lumbar = os.path.join(ruta_procesado, sin_mascara, lumbar, imagenes)
ruta_salida_columna = os.path.join(ruta_procesado, sin_mascara, columna, imagenes)

# Convertimos los ficheros jpg a png
jpg_2_png(ruta_entrada_cervical, ruta_salida_cervical)
jpg_2_png(ruta_entrada_lumbar, ruta_salida_lumbar)
jpg_2_png(ruta_entrada_columna, ruta_salida_columna, False)